# Optimization in JuMP

Code: https://github.com/JuliaOpt/JuMP.jl Docs: http://www.juliaopt.org/JuMP.jl

From the documentation:

"JuMP is a domain-specific modeling language for mathematical optimization embedded in Julia. It currently supports a number of open-source and commercial solvers (see below) for a variety of problem classes, including linear programming, mixed-integer programming, second-order conic programming, semidefinite programming, and nonlinear programming."
* Simple syntax
* Fast
* Solver independent language
* Supports many solvers
 - Including: Artelys Knitro, Bonmin, Cbc, Clp, Couenne, CPLEX, ECOS, FICO Xpress, GLPK, Gurobi, Ipopt, MOSEK, NLopt, and SCS.
* Written in Julia

Note: Not all solvers support all sorts of sets and constraints. This means that you should select a solver after your requirements. But also that you can use very efficient solvers for special problems, such as LP or QP. 

### In JuMP, each model is connected with an optimizer

In [1]:
using JuMP, Ipopt
model = Model(with_optimizer(Ipopt.Optimizer, print_level=0))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: SolverName() attribute not implemented by the optimizer.

Variables are defined using `@variable` and connected to a model.
Vectors can be defined using `x[1:n]`, and simple constraints can optionally be added

In [2]:
n = 10
@variable(model, x[1:n] >= 0)

10-element Array{VariableRef,1}:
 x[1] 
 x[2] 
 x[3] 
 x[4] 
 x[5] 
 x[6] 
 x[7] 
 x[8] 
 x[9] 
 x[10]

### Adding Constraints
Simple constraints can be added uisng the built in Julia operators, and optionally be given names using

`@constraint(model, [name,] constraint)`

or 

`@constraint(model, [name,] expression in MOI.SomeSetHere())`

In [3]:
@constraint(model, firstbig, 1 <= x[10] <= 10)

firstbig : x[10] ∈ [1.0, 10.0]

And a set of constraints can be created in a simple way

In [4]:
@constraint(model, first3[k=1:3], x[k] <= 10*k)

3-element Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}:
 first3[1] : x[1] ≤ 10.0
 first3[2] : x[2] ≤ 20.0
 first3[3] : x[3] ≤ 30.0

Constraints can then be inspected as expected

In [5]:
firstbig

firstbig : x[10] ∈ [1.0, 10.0]

In [6]:
first3

3-element Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}:
 first3[1] : x[1] ≤ 10.0
 first3[2] : x[2] ≤ 20.0
 first3[3] : x[3] ≤ 30.0

And the full model so far can be printed

In [7]:
model

A JuMP Model
Feasibility problem with:
Variables: 10
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 10 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.Interval{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: SolverName() attribute not implemented by the optimizer.
Names registered in the model: first3, firstbig, x

Vector constraints can be added with the broadcast syntax `.`

In [8]:
using Random
Random.seed!(2)
m = 3

A = randn(m,n)
b = randn(m)

@constraint(model, A*x .== b);

Expressions can be built up using `@expression`

In [9]:
C = randn(m,n)
d = randn(m)

ineq = @expression(model, C*x - d);

And add constraints for expresions in sets

In [10]:
@constraint(model, ineq in MOI.Nonnegatives(m));

Or we could simply have written

`@constraint(model, C*x .<= d)`

### Objective
Simple cost functions can be defined by

`@objective(model, sense, expression)`

In [11]:
M = randn(n,n)
Q = M*M'
p = randn(n)

@objective(model, Min,  x'*Q*x - p'*x);

### Solving
the solver is called using `optimize!`
and values are accessed using `value`

In [13]:
optimize!(model)

@show termination_status(model)
sol = value.(x)

termination_status(model) = LOCALLY_SOLVED::TerminationStatusCode = 4


10-element Array{Float64,1}:
 0.14117701606066796
 1.3225117817778225 
 0.0                
 0.44180210934358255
 1.066265309511417  
 2.1971581225582093 
 0.0                
 2.6830282965053636 
 0.0                
 0.999999990015267  

We can also ask the solver for a range of data, such as objective_value, dual variables and so on.

See http://www.juliaopt.org/JuMP.jl for details

In [14]:
@show objective_value(model)

@show A*sol-b         # Should be rougly zero

# We can evaluate expressions
@show value.(ineq);    # Should be nonnegative

objective_value(model) = 153.43269713049497
A * sol - b = [4.89703e-9, 2.11316e-8, 1.05965e-8]
value.(ineq) = [-1.16673e-8, -2.46458e-8, 6.83794]


### General nonlinear programming is also supported by some solvers

These objectives and constraints can be entered as

`@NLobjective(model, Min, my_function(x, y))`

and

`@NLconstraint(model, x[1] * x[2] * x[3] * x[4] >= 25)`